In [68]:
import pandas as pd
import datetime
import plotly as plt
import seaborn as sns

In [69]:
def load_for_matrix():
    global df
    df = pd.read_csv('data/total.csv')
    df.rename(columns= {'Unnamed: 0': 'date'}, inplace=True)
    df.date = pd.to_datetime(df.date)
    df.set_index('date', inplace=True)
    df = df.groupby('day_id').resample('1Min').ffill()
    df = pd.DataFrame(df)
    return df
load_for_matrix()

customer_no  location        day   day_id
day_id  date                                                          
1-fri   2019-09-06 07:00:00            1     dairy     friday    1-fri
        2019-09-06 07:01:00            1     dairy     friday    1-fri
        2019-09-06 07:02:00            1     dairy     friday    1-fri
        2019-09-06 07:03:00            1     dairy     friday    1-fri
        2019-09-06 07:04:00            1    spices     friday    1-fri
...                                  ...       ...        ...      ...
999-tue 2019-09-03 18:00:00          999     dairy    tuesday  999-tue
        2019-09-03 18:01:00          999     dairy    tuesday  999-tue
        2019-09-03 18:02:00          999  checkout    tuesday  999-tue
999-wed 2019-09-04 16:53:00          999     fruit  wednesday  999-wed
        2019-09-04 16:54:00          999  checkout  wednesday  999-wed

[54192 rows x 4 columns]

In [70]:
def generate_prob_matrix(df):
    """This function takes a dataframe with customer movements over time inside the store. It generates a probability matrix of size N x N. where N is the total number of locations in the store. The matrix contains the probabilities that a customer will go to the next location based on the current one within the time defined by the timestep in the dataframe. It is meant to be used further in a Markov chain Monte Carlo simulation.

    INPUT ARGUMENTS:
        -df -> pandas DataFrame with a timedate index, customer_ID and location at every timestep.

    OUTPUT:
        -prob_matrix -> matrix where each element is the probability to transition from one state to another in a Markov chain simulation. 
    """
    
    # create empty series that will be populated with the current and previous locations for each customer
    loc_now = pd.Series(dtype=object)
    loc_before = pd.Series(dtype=object)
    
    # iterate through all customer IDs
    for customer_id in df.day_id.unique():
        
        # find the current and previous locations for just the current customer
        loc_per_customer = df[df['day_id'] ==  customer_id]['location']
        loc_per_customer_before = loc_per_customer.shift(1)
        
        # shifting data will create NaN values for previous locations; they will be replaced with 'entrance'
        loc_per_customer_before.fillna('entrance',
                               inplace=True)
        
        # add the current and previous locations to the series
        loc_now = pd.concat([loc_now,
                            loc_per_customer])
        loc_before = pd.concat([loc_before,
                                loc_per_customer_before])    
    
    # generate transition probability matrix
    prob_matrix = pd.crosstab(loc_before, 
                            loc_now,
                            normalize='index')
    print(prob_matrix)
    return [loc_before, loc_now]

In [71]:
x = generate_prob_matrix(df)

col_0     checkout     dairy    drinks     fruit    spices
row_0                                                     
dairy     0.102697  0.738706  0.058134  0.049478  0.050986
drinks    0.215334  0.010880  0.599199  0.087755  0.086832
entrance  0.000000  0.287576  0.153526  0.377435  0.181464
fruit     0.200564  0.095428  0.054564  0.599029  0.050415
spices    0.149613  0.191839  0.161948  0.090305  0.406294


In [74]:
x = pd.DataFrame(x).transpose()
x.rename(columns={0:'before', 1:'after'}, inplace=True)
x

,before,after
"(1-fri, 2019-09-06 07:00:00)",entrance,dairy
"(1-fri, 2019-09-06 07:01:00)",dairy,dairy
"(1-fri, 2019-09-06 07:02:00)",dairy,dairy
"(1-fri, 2019-09-06 07:03:00)",dairy,dairy
"(1-fri, 2019-09-06 07:04:00)",dairy,spices
...,...,...
"(999-tue, 2019-09-03 18:00:00)",dairy,dairy
"(999-tue, 2019-09-03 18:01:00)",dairy,dairy
"(999-tue, 2019-09-03 18:02:00)",dairy,checkout
"(999-wed, 2019-09-04 16:53:00)",entrance,fruit


In [77]:
for unit in x.items():
    print(unit)

('before', (1-fri, 2019-09-06 07:00:00)      entrance
(1-fri, 2019-09-06 07:01:00)         dairy
(1-fri, 2019-09-06 07:02:00)         dairy
(1-fri, 2019-09-06 07:03:00)         dairy
(1-fri, 2019-09-06 07:04:00)         dairy
                                    ...   
(999-tue, 2019-09-03 18:00:00)       dairy
(999-tue, 2019-09-03 18:01:00)       dairy
(999-tue, 2019-09-03 18:02:00)       dairy
(999-wed, 2019-09-04 16:53:00)    entrance
(999-wed, 2019-09-04 16:54:00)       fruit
Name: before, Length: 54192, dtype: object)
('after', (1-fri, 2019-09-06 07:00:00)         dairy
(1-fri, 2019-09-06 07:01:00)         dairy
(1-fri, 2019-09-06 07:02:00)         dairy
(1-fri, 2019-09-06 07:03:00)         dairy
(1-fri, 2019-09-06 07:04:00)        spices
                                    ...   
(999-tue, 2019-09-03 18:00:00)       dairy
(999-tue, 2019-09-03 18:01:00)       dairy
(999-tue, 2019-09-03 18:02:00)    checkout
(999-wed, 2019-09-04 16:53:00)       fruit
(999-wed, 2019-09-04 16:54:00)  